In [1]:
import arviz as az
import pandas as pd
import json
import os

import numpy as np
import diffrax
import matplotlib.pyplot as plt
import seaborn as sns
import jax
import sys
from scipy.stats import mode

sys.path.append("../models/")
from huang_ferrell_1996 import *
from bhalla_iyengar_1999 import *
from kholodenko_2000 import *
from levchenko_2000 import *
from brightman_fell_2000 import *
from schoeberl_2002 import *
from hatakeyama_2003 import *
from hornberg_2005 import *
from birtwistle_2007 import *
from orton_2009 import *
from vonKriegsheim_2009 import *
from shin_2014 import *
from ryu_2015 import *
from kochanczyk_2017 import *
from dessauges_2022 import *

jax.config.update("jax_enable_x64", True)
%matplotlib inline

sys.path.insert(0, '../')
from utils import *

rng = np.random.default_rng(seed=1234)

# On Martini set the visible GPU to the 2080Ti
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


/home/nlinden/mambaforge/envs/pymc5_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Synthetic Data

In [2]:
# Kholodenko 2000
K_2000_idata, K_2000_ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/kholodenko_2000/kholodenko_2000_smc_samples.json')
K_2000_post = np.load('../../../results/MAPK/param_est/synthetic_data/kholodenko_2000/kholodenko_2000_posterior_predictive_samples.npy')
# Levchenko 2000
L_2000_idata, L_2000_ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/levchenko_2000/levchenko_2000_smc_samples.json')
L_2000_post = np.load('../../../results/MAPK/param_est/synthetic_data/levchenko_2000/levchenko_2000_posterior_predictive_samples.npy')
# Hatakeyama 2003
HK_2003_idata, HK_2003_ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/hatakeyama_2003/hatakeyama_2003_smc_samples.json')
HK_2003_post = np.load('../../../results/MAPK/param_est/synthetic_data/hatakeyama_2003/hatakeyama_2003_posterior_predictive_samples.npy')
# Hornberg 2005
H_2005_idata, H_2005_ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/hornberg_2005/hornberg_2005_smc_samples.json') # TODO: add once smc is done
H_2005_post = np.load('../../../results/MAPK/param_est/synthetic_data/hornberg_2005/hornberg_2005_posterior_predictive_samples.npy')
# Birtwistle 2007
B_2007_idata, B_2007_ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/birtwistle_2007/birtwistle_2007_smc_samples.json')
B_2007_post = np.load('../../../results/MAPK/param_est/synthetic_data/birtwistle_2007/birtwistle_2007_posterior_predictive_samples.npy')
# Orton 2009
O_2009_idata, O_2009_ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/orton_2009/orton_2009_smc_samples.json') # TODO: add once smc is done
O_2009_post = np.load('../../../results/MAPK/param_est/synthetic_data/orton_2009/orton_2009_posterior_predictive_samples.npy')
# von Kreigsheim 2009
# vK_2009_idata, ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/von_kreigsheim_2009/von_kreigsheim_2009_smc_samples.json') # TODO: add once smc is done
# vK_2009_post = np.load('../../../results/MAPK/param_est/synthetic_data/von_kreigsheim_2009/von_kreigsheim_2009_posterior_predictive_samples.npy')
# Shin 2014
S_2014_idata, S_2014_ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/shin_2014/shin_2014_smc_samples.json')
S_2014_post = np.load('../../../results/MAPK/param_est/synthetic_data/shin_2014/shin_2014_posterior_predictive_samples.npy')
# Ryu 2015
R_2015_idata, R_2015_ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/ryu_2015/ryu_2015_smc_samples.json') # TODO: add once smc is done
R_2015_post = np.load('../../../results/MAPK/param_est/synthetic_data/ryu_2015/ryu_2015_posterior_predictive_samples.npy')
# Kochańczyk 2017
K_2017_idata, K_2017_ss = load_smc_samples_to_idata('../../../results/MAPK/param_est/synthetic_data/kochanczyk_2017/kochanczyk_2017_smc_samples.json') # TODO: add once smc is done
K_2017_post = np.load('../../../results/MAPK/param_est/synthetic_data/kochanczyk_2017/kochanczyk_2017_posterior_predictive_samples.npy')

model_names = ['kholodenko_2000', 'levchenko_2000', 'hatakeyama_2003','hornberg_2005', 'birtwistle_2007', 'orton_2009', 'shin_2014', 'ryu_2015', 'kochanczyk_2017']
display_names = ['Kholodenko 2000', 'Levchenko 2000', 'Katakeyama 2003','Hornberg 2005', 'Birtwistle 2007', 'Orton 2009', 'Shin 2014', 'Ryu 2015', 'Kochańczyk 2017']

# gather posterior samples
posterior_samples = [K_2000_post, L_2000_post, HK_2003_post, H_2005_post, B_2007_post, O_2009_post, S_2014_post, R_2015_post, K_2017_post]
shin_id = 6
# shin has 16000 so downsample to 4000
idxs = rng.choice(np.arange(16000), size=4000, replace=False)
posterior_samples[shin_id] = S_2014_post[idxs]

# get data
inputs, data = load_data('../../../results/MAPK/HF_96_synthetic_data.csv')
HF_trajs = np.load('../../../results/MAPK/HF_96_trajs.npy')
HF_traj_times = np.load('../../../results/MAPK/HF_96_traj_times.npy')
# set up a color palette
colors = sns.color_palette('pastel', n_colors=len(model_names)+2)

# sns.color_palette("husl", len(model_names)+2)

#### Remake posterior predictive dose repsonse in a more compact form

In [3]:
fig, ax = plot_stimulus_response_curve_pretty(K_2000_post, data, inputs, input_name='EGF stimulus (nM)', 
                                 output_name='% maximal ERK \n activity', box_color='w', data_color='r',
                                 data_std=0.1, width=1.1, height=1.1, data_marker_size=5.0, scatter_marker_size=0)
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
# ax.set_xlabel(ax.get_xlabel(), fontsize=10.0)
ax.set_xlabel('')
ax.set_ylabel(ax.get_ylabel(), fontsize=10.0)
ax.set_title('Kholodenko 2000', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/K_2000_posterior_predictive.pdf', transparent=True)


fig, ax = plot_stimulus_response_curve_pretty(L_2000_post, data, inputs, input_name='EGF stimulus (nM)', 
                                 output_name='% maximal ERK \n activity', box_color='w', data_color='r',
                                 data_std=0.1, width=1.1, height=1.1, data_marker_size=5.0, scatter_marker_size=0)
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
# ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_yticklabels([])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
# ax.set_xlabel(ax.get_xlabel(), fontsize=10.0)
ax.set_xlabel('')
# ax.set_ylabel(ax.get_ylabel(), fontsize=10.0)
ax.set_ylabel('')
ax.set_title('Levchenko 2000', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/L_2000_posterior_predictive.pdf', transparent=True)

fig, ax = plot_stimulus_response_curve_pretty(HK_2003_post, data, inputs, input_name='EGF stimulus (nM)', 
                                 output_name='% maximal ERK \n activity', box_color='w', data_color='r',
                                 data_std=0.1, width=1.1, height=1.1, data_marker_size=5.0, scatter_marker_size=0)
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
# ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_yticklabels([])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
# ax.set_xlabel(ax.get_xlabel(), fontsize=10.0)
ax.set_xlabel('')
# ax.set_ylabel(ax.get_ylabel(), fontsize=10.0)
ax.set_ylabel('')
ax.set_title('Hatakeyama 2003', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/HK_2003_posterior_predictive.pdf', transparent=True)

# TODO: Hornberg 2005
fig, ax = fig, ax = plot_stimulus_response_curve_pretty(H_2005_post, data, inputs, input_name='EGF stimulus (nM)', 
                                 output_name='% maximal ERK \n activity', box_color='w', data_color='r',
                                 data_std=0.1, width=1.1, height=1.1, data_marker_size=5.0, scatter_marker_size=0)
ax.set_xscale('log')
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
# ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_yticklabels([])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
# ax.set_xlabel(ax.get_xlabel(), fontsize=10.0)
ax.set_xlabel('')
ax.set_xlim([0.0006539278817108092, 0.18412936150507658])
ax.set_ylabel('% maximal ERK \n activity', fontsize=10.0)
ax.set_ylabel('')
ax.set_title('Hornberg 2005', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/H_2005_posterior_predictive.pdf', transparent=True)


fig, ax = plot_stimulus_response_curve_pretty(B_2007_post, data, inputs, input_name='EGF stimulus (nM)', 
                                 output_name='% maximal ERK \n activity', box_color='w', data_color='r',
                                 data_std=0.1, width=1.1, height=1.1, data_marker_size=5.0, scatter_marker_size=0)
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
# ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_yticklabels([])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
# ax.set_xlabel(ax.get_xlabel(), fontsize=10.0)
ax.set_xlabel('')
# ax.set_ylabel(ax.get_ylabel(), fontsize=10.0)
ax.set_ylabel('')
ax.set_title('Birtwistle 2007', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/B_2007_posterior_predictive.pdf', transparent=True)


fig, ax = plot_stimulus_response_curve_pretty(O_2009_post, data, inputs, input_name='EGF stimulus (nM)', 
                                 output_name='% maximal ERK \n activity', box_color='w', data_color='r',
                                 data_std=0.1, width=1.1, height=1.1, data_marker_size=5.0, scatter_marker_size=0)
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
ax.set_xlabel(ax.get_xlabel(), fontsize=10.0)
ax.set_ylabel(ax.get_ylabel(), fontsize=10.0)
ax.set_title('Orton 2009', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/O_2009_posterior_predictive.pdf', transparent=True)

# TODO: von Kriegsheim 2009
fig, ax = get_sized_fig_ax(1.1, 1.1)
ax.set_xscale('log')
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
# ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_yticklabels([])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
ax.set_xlabel('EGF stimulus (nM)', fontsize=10.0)
# ax.set_ylabel('% maximal ERK \n activity', fontsize=10.0)
ax.set_xlim([0.0006539278817108092, 0.18412936150507658])
ax.set_ylabel('')
ax.set_title('von Kriegsheim 2009', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/VK_2009_posterior_predictive.pdf', transparent=True)

fig, ax = plot_stimulus_response_curve_pretty(S_2014_post, data, inputs, input_name='EGF stimulus (nM)', 
                                 output_name='% maximal ERK \n activity', box_color='w', data_color='r',
                                 data_std=0.1, width=1.1, height=1.1, data_marker_size=5.0, scatter_marker_size=0)
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
# ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_yticklabels([])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
ax.set_xlabel(ax.get_xlabel(), fontsize=10.0)
# ax.set_ylabel(ax.get_ylabel(), fontsize=10.0)
ax.set_ylabel('')
ax.set_title('Shin 2014', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/S_2014_posterior_predictive.pdf', transparent=True)

fig, ax = plot_stimulus_response_curve_pretty(R_2015_post, data, inputs, input_name='EGF stimulus (nM)', 
                                 output_name='% maximal ERK \n activity', box_color='w', data_color='r',
                                 data_std=0.1, width=1.1, height=1.1, data_marker_size=5.0, scatter_marker_size=0)
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
# ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_yticklabels([])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
ax.set_xlabel(ax.get_xlabel(), fontsize=10.0)
# ax.set_ylabel(ax.get_ylabel(), fontsize=10.0)
ax.set_ylabel('')
ax.set_title('Ryu 2015', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/R_2015_posterior_predictive.pdf', transparent=True)

fig, ax = plot_stimulus_response_curve_pretty(K_2017_post, data, inputs, input_name='EGF stimulus (nM)', 
                                 output_name='% maximal ERK \n activity', box_color='w', data_color='r',
                                 data_std=0.1, width=1.1, height=1.1, data_marker_size=5.0, scatter_marker_size=0)
ax.set_ylim([-0.25, 1.25])
ax.set_yticks([0, 0.5, 1.0])
# ax.set_yticklabels(['0', '50', '100'], fontsize=8)
ax.set_yticklabels([])
ax.set_xticks([1e-3, 1e-2, 1e-1])
ax.set_xticklabels(ax.get_xticks(), fontsize=8)
ax.set_xlabel(ax.get_xlabel(), fontsize=10.0)
# ax.set_ylabel(ax.get_ylabel(), fontsize=10.0)
ax.set_ylabel('')
ax.set_title('Kochańczyk 2017', fontsize=10.0)
fig.savefig('../../../figures/MAPK/param_est/K_2017_posterior_predictive.pdf', transparent=True)
plt.close('all')

### Generate trajectories for each model
We need to normalize these to some model-specific maximum. For this we will use the max value over all trajectories for each model

In [4]:
data_savedir = '../../../results/MAPK/param_est/synthetic_data/'
fig_savedir = '../../../figures/MAPK/param_est/'
# Kholodenko 2000
K_2000_traj, K_2000_converted_times, K_2000_params = get_trajectories(
    'kholodenko_2000', K_2000_idata, 400, 14400, 1/60, 100, 
    'Input', 1.0, ['MAPK_PP'], inputs[[0,3,9]], data_savedir)

# Levchenko 2000
L_2000_traj, L_2000_converted_times, L_2000_params = get_trajectories(
    'levchenko_2000', L_2000_idata, 400, 14400, 1/60, 100, 'RAFact', 0.001, ['MAPKstarstar'], inputs[[0,3,9]], data_savedir)

# Hatakeyama 2003
HK_2003_traj, HK_2003_converted_times, HK_2003_params = get_trajectories(
    'hatakeyama_2003', HK_2003_idata, 400, 14400, 1/60, 100, 'HRG', 1, ['ERKPP'], inputs[[0,3,9]], data_savedir)

# Hornberg 2005
H_2005_traj, H_2005_converted_times, H_2005_params = get_trajectories(
    'hornberg_2005', H_2005_idata, 400, 14400, 1/60, 100, 'c1', 0.000000001, ['c59','c83'], inputs[[0,3,9]], data_savedir)

# Birtwistle 2007
B_2007_traj, B_2007_converted_times, B_2007_params = get_trajectories(
    'birtwistle_2007', B_2007_idata, 400, 14400, 1/60, 100, 'E', 1.0, 
    ['ERKstar','ERKstar_ERKpase'], inputs[[0,3,9]], data_savedir)

# Orton 2009
O_2009_traj, O_2009_converted_times, O_2009_params = get_trajectories(
    'orton_2009', O_2009_idata, 400, 240, 1, 100,'EGF', 602214, ['ErkActive'],
    inputs[[0,3,9]], data_savedir)

# VonKriegsheim 2009
# TODO: run when inference is complete
# VK_2009_traj, VK_2009_traj_norm, VK_2009_converted_times = get_trajectories(
#     'vonKriegsheim_2009', VK_2009_idata, 400, 240, 1, 100, 'EGF', 6.048, 
#     ['ppERK','ppERK_15','ppERKn'],inputs[[0,3,9]], data_savedir)

# Shin 2014
S_2014_traj, S_2014_converted_times, S_2014_params = get_trajectories(
    'shin_2014', S_2014_idata, 400, 240, 1, 100, 'EGF',  0.001, ['pp_ERK'],
    inputs[[0,3,9]], data_savedir)

# Ryu 2015
R_2015_traj, R_2015_converted_times, R_2015_params = get_trajectories(
    'ryu_2015', R_2015_idata, 400, 240, 1, 100, 'EGF', 6.048, ['ERK_star'], inputs[[0,3,9]], data_savedir)

# Kochanczyk 2017
K_2017_traj,  K_2017_converted_times, K_2017_params = get_trajectories(
    'kochanczyk_2017', K_2017_idata, 400, 14400, 1/60, 100, 'EGF', 6048, ['ERKSPP'],
    inputs[[0,3,9]], data_savedir)

I0000 00:00:1701806160.684085 2615653 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


### Make plots for each model
We will make the following plots:
- Normalized trajectory - spaghetti
- Normalized trajectory - avg + std
- SAM60
- Tmax

In [5]:
# normalize HF_trajs
# HF_trajs_norm = np.apply_along_axis(normalize_traj, 1, np.squeeze(HF_trajs), np.max(HF_trajs))
HF_trajs = np.squeeze(HF_trajs)
HF_trajs_norm = HF_trajs/1.2

In [7]:
# Kholodenko 2000
K_2000_conv = np.tile(K_2000_params[:,-1], (100,1)).T
K_2000_traj_norm = normalize_trajs(K_2000_traj, K_2000_conv, 'kholodenko_2000', data_savedir)
make_traj_plots('K_2000', 'Kholodenko 2000', inputs[[0,3,9]], 400, K_2000_traj_norm, K_2000_converted_times, fig_savedir, HF_trajs[[0,3,9]], HF_traj_times)
plt.close('all')

# Levchenko 2000
L_2000_conv = np.array(0.4)
L_2000_traj_norm = normalize_trajs(L_2000_traj, L_2000_conv, 'levchenko_2000', data_savedir)
make_traj_plots('L_2000', 'Levchenko 2000', inputs[[0,3,9]], 400, L_2000_traj_norm, L_2000_converted_times, fig_savedir, HF_trajs[[0,3,9]])
plt.close('all')

# Hatakeyama 2003
HK_2003_conv = np.array(1000.0)
HK_2003_traj_norm = normalize_trajs(HK_2003_traj, HK_2003_conv, 'hatakeyama_2003', data_savedir)
make_traj_plots('HK_2003', 'Hatakeyama 2003', inputs[[0,3,9]], 400,  HK_2003_traj_norm, HK_2003_converted_times, fig_savedir, HF_trajs[[0,3,9]], HF_traj_times)
plt.close('all')

# Hornberg 2005
H_2005_conv = np.array(22100000.0)
H_2005_traj_norm = normalize_trajs(H_2005_traj, H_2005_conv, 'hornberg_2005', data_savedir)
make_traj_plots('H_2005', 'Hornberg 2005', inputs[[0,3,9]], 400,H_2005_traj_norm, H_2005_converted_times, fig_savedir, HF_trajs[[0,3,9]])
plt.close('all')

# Birtwistle 2007
B_2007_conv = np.array(750.0)
B_2007_traj_norm = normalize_trajs(B_2007_traj, B_2007_conv, 'birtwistle_2007', data_savedir)
make_traj_plots('B_2007', 'Birtwistle 2007', inputs[[0,3,9]], 400,  B_2007_traj_norm, B_2007_converted_times, fig_savedir, HF_trajs[[0,3,9]])
plt.close('all')

# Orton 2009
O_2009_conv = np.array(600000.0)
O_2009_traj_norm = normalize_trajs(O_2009_traj, O_2009_conv, 'orton_2009', data_savedir)
make_traj_plots('O_2009', 'Orton 2009', inputs[[0,3,9]], 400, O_2009_traj_norm, O_2009_converted_times, fig_savedir, HF_trajs[[0,3,9]])
plt.close('all')

# # vonKriegsheim 2009
## VK_2009_conv = np.array(0.0311267292018+0.9688732707982)
## make_traj_plots('VK_2009', 'vonKriegsheim 2009', inputs[[0,3,9]], 400, VK_2009_traj, VK_2009_converted_times, fig_savedir, HF_trajs[[0,3,9]], VK_2009_conv)
##plt.close('all')

# Shin 2014
S_2014_conv = np.tile(S_2014_params[:,6], (100,1)).T
S_2014_traj_norm = normalize_trajs(S_2014_traj, S_2014_conv, 'shin_2014', data_savedir)
make_traj_plots('S_2014', 'Shin 2014', inputs[[0,3,9]], 400, S_2014_traj_norm, S_2014_converted_times, fig_savedir, HF_trajs[[0,3,9]])
#plt.close('all')

# Ryu 2015
R_2015_conv = np.array(1.0) # this model as arbitrary units...
R_2015_traj_norm = normalize_trajs(R_2015_traj, R_2015_conv, 'ryu_2015', data_savedir)
make_traj_plots('R_2015', 'Ryu 2015', inputs[[0,3,9]], 400, R_2015_traj_norm, R_2015_converted_times, fig_savedir, HF_trajs[[0,3,9]])
plt.close('all')

# Kochanczyk 2017
K_2017_conv = np.array(3000000.0)
K_2017_traj_norm = normalize_trajs(K_2017_traj, K_2017_conv, 'kochanczyk_2017', data_savedir)
make_traj_plots('K_2017', 'Kochanczyk 2017', inputs[[0,3,9]], 400, K_2017_traj_norm, K_2017_converted_times, fig_savedir, HF_trajs[[0,3,9]],)
plt.close('all')

NameError: name 'L_2000_traj' is not defined